In [3]:
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "You are a helpful assistant that defines words. Define this word: {word}."
)

llm = GPT4All(
    model="./falcon.bin",
)

chain = prompt | llm

chain.invoke({"word": "tomato"})

'\nA tomato is a fruit that is typically red, round, and has a juicy flesh inside. It is often used in cooking and can be eaten raw or cooked.'